<a href="https://colab.research.google.com/github/JPengine/Asymmetric-regression/blob/main/Asy_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy
import scipy.special as sp
from statsmodels import api
from scipy.stats import laplace_asymmetric
from google.colab import files
import seaborn as sns
from scipy import stats
import scipy.stats
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def skew_regression(dist, y, x=None, initial=None, method="Nelder-Mead", bounds=None, options=None):

    # Bounds for betas and parameters for each distribution
    bd_SN = [(-np.inf, np.inf), (-np.inf, np.inf), (0, np.inf), (-np.inf, np.inf)]
    bd_T = [(-np.inf, np.inf), (-np.inf, np.inf), (0, np.inf), (-np.inf, np.inf), (0, np.inf)]
    bd_SHS1 = [(-np.inf, np.inf), (-np.inf, np.inf), (0, np.inf), (0, np.inf)]
    bd_SHS2 = [(-np.inf, np.inf), (-np.inf, np.inf), (0, np.inf), (-1, 1)]

    b0, b1, sigma, nu, _lambda, alpha = None, None, None, None, None, None

    if len(y) == 0:
        raise ValueError("y must be provided")

    if x is None:
        x = np.ones(len(y)).reshape(-1, 1)

    # Initial values
    sigma = 0.5
    nu = 6
    alpha=0.3
    our_lambda=2

    n = y.shape[0]
    p = x.shape[1]

    if dist in ['SN', 'ST', 'SHS1', 'SHS2']:

        # Define the likelihood function based on the distribution
        if dist == 'SN':
            def LL(b):
                R = (2 / b[p]) * scipy.stats.norm.pdf((y - np.dot(x, b[:p])) / b[p]) * \
                    scipy.stats.norm.cdf(b[p+1] * (y - np.dot(x, b[:p])) / b[p])
                return -np.sum(np.log(R))

            if initial is None:
                initial = np.concatenate((para_ini, [sigma, our_lambda]))

            if bounds is None:
                bounds = bd_SN

        elif dist == 'ST':
            def LL(b):
                R = 2/b[p] * scipy.stats.t.pdf(x=(y - np.dot(x, b[:p])) / b[p], df=b[p+2]) * \
                    scipy.stats.t.cdf((b[p+1]*(y - np.dot(x, b[:p])) / b[p]) * \
                    np.sqrt((b[p+2]+1)/(b[p+2]+((y - np.dot(x, b[:p])) / b[p])**2/b[p]**2)), df=b[p+2] + 1)
                return -np.sum(np.log(R))

            if initial is None:
                initial = np.concatenate((para_ini, [sigma, our_lambda, nu]))

            if bounds is None:
                bounds = bd_T

        elif dist == 'SHS2':
            def LL(b):
                t1 = np.cos((np.pi/2) * b[p+1]) / (np.pi * b[p+1])
                t2 = np.exp(b[p+1] * ((y - np.dot(x, b[:p])) / b[p])) / \
                     (np.cosh((y - np.dot(x, b[0:p])) / b[p]))
                R = t1 * t2
                return -np.sum(np.log(R))

            if initial is None:
                initial = np.concatenate((para_ini, [sigma, alpha]))

            if bounds is None:
                bounds = bd_SHS2

        elif dist == 'SHS1':
            def LL(b):
                R = (2 * sp.gamma((b[p+1]+1)/2) / (b[p] * np.sqrt(b[p+1] * np.pi) * sp.gamma(b[p+1]/2))) * \
                    np.exp((y - np.dot(x, b[0:p])) / b[p]) * \
                    (1 + np.exp(2*((y - np.dot(x, b[0:p])) / b[p])) / b[p+1])**-((b[p+1]+1)/2)
                return -np.sum(np.log(R))

            if initial is None:
                initial = np.concatenate((para_ini, [sigma, nu]))

            if bounds is None:
                bounds = bd_SHS1

        # Perform the optimization
        fit = minimize(LL, initial, method=method, bounds=bounds, options=options)

        # Extract optimized parameters based on distribution type
        coefficient = fit.x[0:p]
        if dist == 'SN':
            b0, b1 = coefficient
            sigma = fit.x[p]
            _lambda = fit.x[p+1]
            return b0, b1, sigma, _lambda
        elif dist == 'ST':
            b0, b1 = coefficient
            sigma = fit.x[p]
            _lambda = fit.x[p+1]
            nu = fit.x[p+2]
            return b0, b1, sigma, _lambda, nu
        elif dist == 'SHS2':
            b0, b1 = coefficient
            sigma = fit.x[p]
            alpha = fit.x[p+1]
            return b0, b1, sigma, alpha
        elif dist == 'SHS1':
            b0, b1 = coefficient
            sigma = fit.x[p]
            nu = fit.x[p+1]
            return b0, b1, sigma, nu

    else:
        raise ValueError("Invalid distribution type. Choose 'SN' for skew normal, 'ST' for student's t, 'SHS1' for skew hyperbolic secant I, or 'SHS2' for skew hyperbolic secant II.")